# 📊 Aadhaar Data Cleaning & Preparation

**Objective**: Load, merge, clean, and validate all Aadhaar datasets for analysis.

**Datasets**:
- **Enrollment**: New Aadhaar registrations by age group
- **Demographic Updates**: Citizen demographic data corrections
- **Biometric Updates**: Fingerprint/iris data refresh requests

---

## 1️⃣ Environment Setup

In [1]:
import pandas as pd
import glob
import os
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ Libraries loaded successfully")
print(f"Pandas version: {pd.__version__}")

✅ Libraries loaded successfully
Pandas version: 2.3.3


## 2️⃣ Data Loading (Multi-CSV Merge)

Using `glob` to load all CSV files from each data folder and concatenate into unified DataFrames.

In [2]:
# Load all enrollment CSVs
enrollment_files = glob.glob("../data/enrollment/*.csv")
print(f"📁 Found {len(enrollment_files)} enrollment files:")
for f in enrollment_files:
    print(f"   - {os.path.basename(f)}")

enrol = pd.concat(
    [pd.read_csv(f) for f in enrollment_files],
    ignore_index=True
)
print(f"\n✅ Enrollment DataFrame created: {enrol.shape[0]:,} rows × {enrol.shape[1]} columns")

📁 Found 3 enrollment files:
   - api_data_aadhar_enrolment_0_500000.csv
   - api_data_aadhar_enrolment_1000000_1006029.csv
   - api_data_aadhar_enrolment_500000_1000000.csv



✅ Enrollment DataFrame created: 1,006,029 rows × 7 columns


In [3]:
# Load all demographic CSVs
demographic_files = glob.glob("../data/demographic/*.csv")
print(f"📁 Found {len(demographic_files)} demographic files:")
for f in demographic_files:
    print(f"   - {os.path.basename(f)}")

demo = pd.concat(
    [pd.read_csv(f) for f in demographic_files],
    ignore_index=True
)
print(f"\n✅ Demographic DataFrame created: {demo.shape[0]:,} rows × {demo.shape[1]} columns")

📁 Found 5 demographic files:
   - api_data_aadhar_demographic_0_500000.csv
   - api_data_aadhar_demographic_1000000_1500000.csv
   - api_data_aadhar_demographic_1500000_2000000.csv
   - api_data_aadhar_demographic_2000000_2071700.csv
   - api_data_aadhar_demographic_500000_1000000.csv



✅ Demographic DataFrame created: 2,071,700 rows × 6 columns


In [4]:
# Load all biometric CSVs
biometric_files = glob.glob("../data/biometric/*.csv")
print(f"📁 Found {len(biometric_files)} biometric files:")
for f in biometric_files:
    print(f"   - {os.path.basename(f)}")

bio = pd.concat(
    [pd.read_csv(f) for f in biometric_files],
    ignore_index=True
)
print(f"\n✅ Biometric DataFrame created: {bio.shape[0]:,} rows × {bio.shape[1]} columns")

📁 Found 4 biometric files:
   - api_data_aadhar_biometric_0_500000.csv
   - api_data_aadhar_biometric_1000000_1500000.csv
   - api_data_aadhar_biometric_1500000_1861108.csv
   - api_data_aadhar_biometric_500000_1000000.csv



✅ Biometric DataFrame created: 1,861,108 rows × 6 columns


## 3️⃣ Schema Validation

In [5]:
print("📋 ENROLLMENT SCHEMA:")
print(enrol.dtypes)
print("\n" + "="*50)
print("\n📋 DEMOGRAPHIC SCHEMA:")
print(demo.dtypes)
print("\n" + "="*50)
print("\n📋 BIOMETRIC SCHEMA:")
print(bio.dtypes)

📋 ENROLLMENT SCHEMA:
date              object
state             object
district          object
pincode            int64
age_0_5            int64
age_5_17           int64
age_18_greater     int64
dtype: object


📋 DEMOGRAPHIC SCHEMA:
date             object
state            object
district         object
pincode           int64
demo_age_5_17     int64
demo_age_17_      int64
dtype: object


📋 BIOMETRIC SCHEMA:
date            object
state           object
district        object
pincode          int64
bio_age_5_17     int64
bio_age_17_      int64
dtype: object


## 4️⃣ Date Parsing & Cleaning

**Critical**: Using `errors='coerce'` to safely handle any malformed dates (including Excel `########` artifacts). Using `dayfirst=True` for DD-MM-YYYY format.

In [6]:
# Store original counts for validation
original_counts = {
    'enrollment': len(enrol),
    'demographic': len(demo),
    'biometric': len(bio)
}

# Safe date parsing with errors='coerce'
enrol['date'] = pd.to_datetime(enrol['date'], errors='coerce', dayfirst=True)
demo['date'] = pd.to_datetime(demo['date'], errors='coerce', dayfirst=True)
bio['date'] = pd.to_datetime(bio['date'], errors='coerce', dayfirst=True)

# Check for null dates BEFORE dropping
print("📊 NULL DATE COUNTS (before cleaning):")
print(f"   Enrollment: {enrol['date'].isna().sum():,} null dates")
print(f"   Demographic: {demo['date'].isna().sum():,} null dates")
print(f"   Biometric: {bio['date'].isna().sum():,} null dates")

📊 NULL DATE COUNTS (before cleaning):
   Enrollment: 0 null dates
   Demographic: 0 null dates
   Biometric: 0 null dates


In [7]:
# Drop rows with invalid dates
enrol = enrol.dropna(subset=['date'])
demo = demo.dropna(subset=['date'])
bio = bio.dropna(subset=['date'])

# Calculate rows dropped
print("🗑️ ROWS DROPPED (due to invalid dates):")
print(f"   Enrollment: {original_counts['enrollment'] - len(enrol):,} rows dropped")
print(f"   Demographic: {original_counts['demographic'] - len(demo):,} rows dropped")
print(f"   Biometric: {original_counts['biometric'] - len(bio):,} rows dropped")

🗑️ ROWS DROPPED (due to invalid dates):
   Enrollment: 0 rows dropped
   Demographic: 0 rows dropped
   Biometric: 0 rows dropped


## 5️⃣ Data Validation & Summary

In [8]:
print("="*60)
print("✅ FINAL CLEANED DATASETS")
print("="*60)

print(f"\n📊 ENROLLMENT ({len(enrol):,} records)")
print(f"   Date Range: {enrol['date'].min().strftime('%Y-%m-%d')} to {enrol['date'].max().strftime('%Y-%m-%d')}")
print(f"   Unique States: {enrol['state'].nunique()}")
print(f"   Unique Districts: {enrol['district'].nunique()}")

print(f"\n📊 DEMOGRAPHIC ({len(demo):,} records)")
print(f"   Date Range: {demo['date'].min().strftime('%Y-%m-%d')} to {demo['date'].max().strftime('%Y-%m-%d')}")
print(f"   Unique States: {demo['state'].nunique()}")
print(f"   Unique Districts: {demo['district'].nunique()}")

print(f"\n📊 BIOMETRIC ({len(bio):,} records)")
print(f"   Date Range: {bio['date'].min().strftime('%Y-%m-%d')} to {bio['date'].max().strftime('%Y-%m-%d')}")
print(f"   Unique States: {bio['state'].nunique()}")
print(f"   Unique Districts: {bio['district'].nunique()}")

✅ FINAL CLEANED DATASETS

📊 ENROLLMENT (1,006,029 records)
   Date Range: 2025-03-02 to 2025-12-31
   Unique States: 55
   Unique Districts: 985

📊 DEMOGRAPHIC (2,071,700 records)
   Date Range: 2025-03-01 to 2025-12-29


   Unique States: 65
   Unique Districts: 983

📊 BIOMETRIC (1,861,108 records)
   Date Range: 2025-03-01 to 2025-12-29


   Unique States: 57
   Unique Districts: 974


In [9]:
print("\n📋 ENROLLMENT SAMPLE:")
display(enrol.head(3))

print("\n📋 DEMOGRAPHIC SAMPLE:")
display(demo.head(3))

print("\n📋 BIOMETRIC SAMPLE:")
display(bio.head(3))


📋 ENROLLMENT SAMPLE:


,date,state,district,pincode,age_0_5,age_5_17,age_18_greater
0,2025-03-02,Meghalaya,East Khasi Hills,793121,11,61,37
1,2025-03-09,Karnataka,Bengaluru Urban,560043,14,33,39
2,2025-03-09,Uttar Pradesh,Kanpur Nagar,208001,29,82,12



📋 DEMOGRAPHIC SAMPLE:


,date,state,district,pincode,demo_age_5_17,demo_age_17_
0,2025-03-01,Uttar Pradesh,Gorakhpur,273213,49,529
1,2025-03-01,Andhra Pradesh,Chittoor,517132,22,375
2,2025-03-01,Gujarat,Rajkot,360006,65,765



📋 BIOMETRIC SAMPLE:


,date,state,district,pincode,bio_age_5_17,bio_age_17_
0,2025-03-01,Haryana,Mahendragarh,123029,280,577
1,2025-03-01,Bihar,Madhepura,852121,144,369
2,2025-03-01,Jammu and Kashmir,Punch,185101,643,1091


In [10]:
print("\n✅ DATE DTYPE VALIDATION:")
print(f"   Enrollment 'date' dtype: {enrol['date'].dtype}")
print(f"   Demographic 'date' dtype: {demo['date'].dtype}")
print(f"   Biometric 'date' dtype: {bio['date'].dtype}")


✅ DATE DTYPE VALIDATION:
   Enrollment 'date' dtype: datetime64[ns]
   Demographic 'date' dtype: datetime64[ns]
   Biometric 'date' dtype: datetime64[ns]


## 6️⃣ Summary Statistics

In [11]:
print("📊 ENROLLMENT SUMMARY STATISTICS:")
display(enrol[['age_0_5', 'age_5_17', 'age_18_greater']].describe())

📊 ENROLLMENT SUMMARY STATISTICS:


,age_0_5,age_5_17,age_18_greater
count,1.006029e+06,1.006029e+06,1.006029e+06
mean,3.525709e+00,1.710074e+00,1.673441e-01
std,1.753851e+01,1.436963e+01,3.220525e+00
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000e+00,0.000000e+00,0.000000e+00
50%,2.000000e+00,0.000000e+00,0.000000e+00
75%,3.000000e+00,1.000000e+00,0.000000e+00
max,2.688000e+03,1.812000e+03,8.550000e+02


In [12]:
print("📊 DEMOGRAPHIC UPDATE SUMMARY STATISTICS:")
display(demo[['demo_age_5_17', 'demo_age_17_']].describe())

📊 DEMOGRAPHIC UPDATE SUMMARY STATISTICS:


,demo_age_5_17,demo_age_17_
count,2.071700e+06,2.071700e+06
mean,2.347552e+00,2.144701e+01
std,1.490355e+01,1.252498e+02
min,0.000000e+00,0.000000e+00
25%,0.000000e+00,2.000000e+00
50%,1.000000e+00,6.000000e+00
75%,2.000000e+00,1.500000e+01
max,2.690000e+03,1.616600e+04


In [13]:
print("📊 BIOMETRIC UPDATE SUMMARY STATISTICS:")
display(bio[['bio_age_5_17', 'bio_age_17_']].describe())

📊 BIOMETRIC UPDATE SUMMARY STATISTICS:


,bio_age_5_17,bio_age_17_
count,1.861108e+06,1.861108e+06
mean,1.839058e+01,1.909413e+01
std,8.370421e+01,8.806502e+01
min,0.000000e+00,0.000000e+00
25%,1.000000e+00,1.000000e+00
50%,3.000000e+00,4.000000e+00
75%,1.100000e+01,1.000000e+01
max,8.002000e+03,7.625000e+03


---

## 📋 Data Quality Insights

### Key Findings:

1. **Data Completeness**: All three datasets have been successfully merged from multiple source files, with minimal data loss due to invalid dates.

2. **Temporal Coverage**: The data spans a consistent time period across enrollment, demographic, and biometric datasets, enabling meaningful cross-dataset analysis.

3. **Geographic Granularity**: The data provides district-level granularity across all states and union territories, supporting localized policy analysis.

4. **Data Integrity**: Safe date parsing with `errors='coerce'` ensured that malformed date entries (common in government data exports) were handled gracefully without causing pipeline failures.

### Policy Relevance:

> The cleaned datasets are now ready for analytical workflows. The district-level granularity enables identification of regional enrollment patterns and update hotspots, which can inform targeted awareness campaigns and infrastructure deployment decisions by UIDAI.

---

In [14]:
print("\n" + "="*60)
print("✅ DATA CLEANING COMPLETE")
print("="*60)
print("\nProceeding to analysis notebooks...")
print("Each subsequent notebook will load data independently.")


✅ DATA CLEANING COMPLETE

Proceeding to analysis notebooks...
Each subsequent notebook will load data independently.
